In [1]:
## A note on the below
## This is a monstrosity. I'm not proud of it. I was having fun with exec and eval.


###### NOTEBOOK INPUTS #####
dataset_names = ["MNIST"] #Need to include "CelebA", "CIFAR10", "MNIST"
model_names = ["VAE", "BetaVAE", "DVAE"]  #"VAE","BetaVAE","DVAE", "CRVAE"
output_dir = "/home/everett/data/10-708-project-results/trained_models/"       # the name of the google drive for saving
s3_bucket = "s3://10-708-group-project/trained_models/"
learning_rate=1e-3         # Need to get SOTA LR or do hyperparameter optimization
batch_size=100             # Same as LR
num_epochs=100             # Same as LR


In [ ]:
for model_name in model_names:
  exec(f"from pythae.models import {model_name}, {model_name}Config")
from pythae.trainers import BaseTrainingConfig
from pythae.pipelines.training import TrainingPipeline
for dataset_name in dataset_names:
  exec(f"from pythae.models.nn.benchmarks.{''.join(i.lower() for i in dataset_name if not i.isdigit())} import Encoder_VAE_{''.join(i.upper() for i in dataset_name if not i.isdigit())}, Decoder_AE_{''.join(i.upper() for i in dataset_name if not i.isdigit())}")

In [ ]:
import os

In [ ]:
!nvidia-smi

In [ ]:
import torch
import torchvision.datasets as datasets


In [ ]:
# loads each dataset in the list of dataset names
# maybe eval is bad but hopefully it works
for dataset_name in dataset_names:
  print(f"loading {dataset_name}")
  if dataset_name != "CelebA":
    trainset = eval(f"datasets.{dataset_name}(root=\'~/data\', train=True, download=True, transform=None)")
  else: #CelebA takes a different format
    trainset = eval(f"datasets.{dataset_name}(root=\'~/data\', split=\'train\', download=True, transform=None)")
  data = trainset.data
  num_train = int(data.shape[0] * 0.80)
  num_eval = data.shape[0] - num_train
  if len(data.shape) == 3:
    [H, W] = data.shape[-2:]
    data = data.reshape(-1, 1, H, W)
  assert(len(data.shape) == 4)
  if data.shape[-1] == 3:
    data = data.transpose(0,3,1,2)
  assert(data.max()>1)
  data = data / 255

  exec(f"{dataset_name}_train_dataset = data[:-num_eval]")
  exec(f"{dataset_name}_eval_dataset = data[-num_eval:]")
  


In [ ]:
#train all the models on all the datasets
for model_name in model_names:
  exec(f"""{model_name.lower()}_model_config = {model_name}Config(
      input_dim=(1, 28, 28),
      latent_dim=16
    )""")
  for dataset_name in dataset_names:
    training_config = BaseTrainingConfig(
      output_dir=os.path.join(output_dir, f"{dataset_name}_{model_name}"),
      learning_rate=learning_rate,        
      batch_size=batch_size,
      num_epochs=num_epochs,
    )
    print(f"dataset = {dataset_name}, model = {model_name}")

    exec(f"""{model_name.lower()}_{dataset_name.lower()} = {model_name}(
      model_config={model_name.lower()}_model_config,
      encoder=Encoder_VAE_{''.join(i.upper() for i in dataset_name if not i.isdigit())}({model_name.lower()}_model_config),
      decoder=Decoder_AE_{''.join(i.upper() for i in dataset_name if not i.isdigit())}({model_name.lower()}_model_config))""")

    exec(f"""{model_name.lower()}_{dataset_name.lower()}_pipeline = TrainingPipeline(
      training_config=training_config,
      model = {model_name.lower()}_{dataset_name.lower()}
    )""")

    exec(f"""{model_name.lower()}_{dataset_name.lower()}_pipeline(
      train_data={dataset_name}_train_dataset,
      eval_data={dataset_name}_eval_dataset
    )""")